In [1]:
import requests
import json
import pandas as pd
import numpy as np
from pandas import json_normalize

In [2]:
url = 'https://finance.naver.com/api/sise/etfItemList.nhn'
json_data = json.loads(requests.get(url).text)
df = json_normalize(json_data['result']['etfItemList'])

## 1. 전처리
#### 문자열 처리 쉽도록 종목코드 앞에 'A' 붙임

In [3]:
df['itemcode'] = 'A' + df['itemcode']
df = df.sort_values(by='etfTabCode', axis=0)

In [14]:
df['exchange_nm'] = 'KRX'

In [5]:
df = df.rename(columns = {'itemcode':'code', 'itemname':'name', 'nowVal':'price', 'quant':'volumne_sh', 'amonut':'trading_amt_mln', 'marketSum':'market_cap_100m'})
df = df.drop(['etfTabCode', 'risefall', 'changeVal', 'changeRate', 'threeMonthEarnRate'], axis=1)

In [6]:
df

,itemcode,etfTabCode,itemname,price,nav,volumne_sh,trading_amt_mln,market_cap_100m
0,A069500,1,KODEX 200,42095,42105.0,5426223,228445,51545
242,A237350,1,KODEX 코스피100,32540,32675.0,827,26,146
60,A332930,1,HANARO 200TR,49275,49267.0,563,27,1651
59,A270810,1,KBSTAR 코스닥150,13790,13920.0,7118,97,1669
151,A304770,1,HANARO 코스닥150,13915,14017.0,617,8,351
...,...,...,...,...,...,...,...,...
358,A261110,7,TIGER 미국달러선물레버리지,9280,9279.0,709,6,74
289,A139660,7,KOSEF 미국달러선물인버스,10640,10639.0,53,0,111
270,A253290,7,KBSTAR 헬스케어채권혼합,12410,12406.0,7125,88,124
300,A352560,7,KODEX 다우존스미국리츠(H),11570,NaN,1696,19,104


# 2. 종목 필터링

## 1) 투자목적 적합성
### (1) 레버리지, 인버스 배제
- 종목명에 금지어가 포함돼 있으면 유니버스에서 원천 삭제(`restricted_tokens`)
- 레버리지, 인버스는 단기 투자목적에만 적합하므로 배제

In [7]:
restricted_tokens = ['레버리지', '인버스', '2X']
df = df[~df.itemname.str.contains('|'.join(restricted_tokens))]

## 2) 유동성
### (1) 시총/거래량
- 시총(`limit_market_cap`) >= 300억원
- 거래금액(`limit_trading_amt`) >= 1억원

In [8]:
limit_market_cap = 300    # 300억원
limit_trading_amt = 100   # 1억원 (100백만원)

In [9]:
df = df[np.logical_and(df.market_cap_100m >= limit_market_cap, df.trading_amt_mln >= limit_trading_amt)]

# 3. 파일 저장

In [11]:
df.reset_index(inplace=True, drop=True)

In [13]:
df.to_csv('universe.csv', encoding='utf-8-sig', index=False)